### Necessary imports

In [1]:
!pip install -q -U torch datasets transformers tensorflow langchain playwright html2text sentence_transformers faiss-cpu
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.


### Dependencies

In [2]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)
from datasets import load_dataset
from peft import LoraConfig, PeftModel

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

/home//miniconda3/envs/torch2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-03 16:04:57.349911: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-03 16:04:57.349947: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-03 16:04:57.350785: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-03 16:04:57.354648: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binar

### Load quantized Mistal 7B

In [3]:
#################################################################
# Tokenizer
#################################################################

model_name='mistralai/Mistral-7B-Instruct-v0.1'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

tokenizer_config.json: 100%|██████████| 1.47k/1.47k [00:00<00:00, 345kB/s]
tokenizer.model: 100%|██████████| 493k/493k [00:00<00:00, 28.4MB/s]
tokenizer.json: 100%|██████████| 1.80M/1.80M [00:00<00:00, 3.84MB/s]
special_tokens_map.json: 100%|██████████| 72.0/72.0 [00:00<00:00, 106kB/s]


Your GPU supports bfloat16: accelerate training with bf16=True


config.json: 100%|██████████| 571/571 [00:00<00:00, 151kB/s]
model.safetensors.index.json: 100%|██████████| 25.1k/25.1k [00:00<00:00, 25.4MB/s]
model-00001-of-00002.safetensors: 100%|██████████| 9.94G/9.94G [05:29<00:00, 30.2MB/s]
model-00002-of-00002.safetensors: 100%|██████████| 4.54G/4.54G [01:36<00:00, 47.1MB/s]
generation_config.json: 100%|██████████| 116/116 [00:00<00:00, 33.8kB/s]


### Count number of trainable parameters

In [17]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 262410240
all model parameters: 3752071168
percentage of trainable model parameters: 6.99%


### Build Mistral text generation pipeline

In [5]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.3,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=200,
)

In [6]:
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

### Load and chunk documents. Load chunked documents into FAISS index 

In [15]:
!playwright install 
!playwright install-deps

Installing dependencies...
Switching to root user to install dependencies...
[sudo] password for : 
Traceback (most recent call last):
  File "/home//miniconda3/envs/torch2/bin/playwright", line 8, in <module>
    sys.exit(main())
  File "/home//miniconda3/envs/torch2/lib/python3.9/site-packages/playwright/__main__.py", line 23, in main
    completed_process = subprocess.run(
  File "/home//miniconda3/envs/torch2/lib/python3.9/subprocess.py", line 507, in run
    stdout, stderr = process.communicate(input, timeout=timeout)
  File "/home//miniconda3/envs/torch2/lib/python3.9/subprocess.py", line 1126, in communicate
    self.wait()
  File "/home//miniconda3/envs/torch2/lib/python3.9/subprocess.py", line 1189, in wait
    return self._wait(timeout=timeout)
  File "/home//miniconda3/envs/torch2/lib/python3.9/subprocess.py", line 1933, in _wait
    (pid, sts) = self._try_wait(0)
  File "/home//miniconda3/envs/torch2/lib/python3.9/subprocess.py", line 1891, in _try_wait
    (pid, sts) = os.

In [20]:
import nest_asyncio
nest_asyncio.apply()

# Articles to index
articles = ["https://www..com/blog/the-need-for-speed/",
            "https://www..com/blog/improve-your-green-reading-with-these-easy-steps/",
            "https://www..com/blog/make-the-most-out-of-your--practice/",
            "https://www..com/products/indoor/features/"]

# Scrapes the blogs above
loader = AsyncChromiumLoader(articles)
docs = loader.load()

In [21]:
# Converts HTML to plain text 
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

# Chunk text
text_splitter = CharacterTextSplitter(chunk_size=100, 
                                      chunk_overlap=0)
chunked_documents = text_splitter.split_documents(docs_transformed)

# Load chunked documents into the FAISS index
db = FAISS.from_documents(chunked_documents, 
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

retriever = db.as_retriever()

Created a chunk of size 671, which is longer than the specified 100
Created a chunk of size 751, which is longer than the specified 100
Created a chunk of size 127, which is longer than the specified 100
Created a chunk of size 725, which is longer than the specified 100
Created a chunk of size 163, which is longer than the specified 100
Created a chunk of size 403, which is longer than the specified 100
Created a chunk of size 344, which is longer than the specified 100
Created a chunk of size 487, which is longer than the specified 100
Created a chunk of size 381, which is longer than the specified 100
Created a chunk of size 528, which is longer than the specified 100
Created a chunk of size 103, which is longer than the specified 100
Created a chunk of size 441, which is longer than the specified 100
Created a chunk of size 827, which is longer than the specified 100
Created a chunk of size 820, which is longer than the specified 100
Created a chunk of size 196, which is longer tha

### Create PromptTemplate and LLMChain

In [22]:
prompt_template = """
### [INST] Instruction: Answer the question as an indoor  coach. Here is context to help:

{context}

### QUESTION:
{question} [/INST]
 """

# Create prompt from prompt template 
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain 
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

In [24]:
llm_chain.invoke({"context": "", "question": "what are main aspects of ?"})

/home//miniconda3/envs/torch2/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'context': '',
 'question': 'what are main aspects of ?',
 'text': "\nAs an indoor  coach, I would say that there are several key aspects to focus on when practicing  indoors. These include:\n\n1. Proper stance and posture: It's important to stand with your feet shoulder-width apart and your body weight evenly distributed between your feet. Your knees should be slightly bent and your back straight.\n2. Correct grip: Hold the putter with your dominant hand on the top of the shaft and your non-dominant hand on the bottom. Make sure your grip is firm but not too tight.\n3. Smooth swing: Practice making smooth, consistent swings with your putter. Focus on keeping your arms and shoulders relaxed and your body moving in a fluid motion.\n4. Aim and target: When , it's important to aim for the center of the green and keep your eye on the ball as you swing. This will help you make more accurate shots.\n5."}

### Build RAG Chain

In [25]:
rag_chain = ( 
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

result = rag_chain.invoke("what are the best drills for green reading?")

/home//miniconda3/envs/torch2/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [26]:
result['context']

[Document(page_content='* Practice\n* References\n* Contact\n* Get Your Own \n* * #  Improve your green reading with these easy steps', metadata={'source': 'https://www..com/blog/improve-your-green-reading-with-these-easy-steps/'}),
 Document(page_content='3 easy ways to improve your green reading', metadata={'source': 'https://www..com/blog/improve-your-green-reading-with-these-easy-steps/'}),
 Document(page_content='Now, that you have figured out which type of player you are, we can shift the\nfocus to improving your green reading skills. By following these three simple\nsteps, you can become a better green reader no matter if you are a straight\nline or a curved line thinker. Make sure, that you incorporate those tips\nregularly into your practice and you will see quick results.', metadata={'source': 'https://www..com/blog/improve-your-green-reading-with-these-easy-steps/'}),
 Document(page_content='As easy and boring as this might seem to you, inside those 10 ft you can still\nface

In [27]:
print(result['text'])

 As an indoor  coach, I would recommend practicing the following drills to improve your green reading skills:

  1. Break Drill: This drill involves hitting putts with varying degrees of break from different angles and distances. The goal is to learn how to read and adjust for the break in your putts. You can use a  system to analyze your shot patterns and identify areas where you need to improve.
   
  2. Bias Drill: This drill involves hitting putts with different biases (left-to-right or right-to-left) to learn how to read and adjust for your personal bias. You can use a  system to analyze your shot patterns and identify areas where you need to improve.
   
  3. Speed Drill: This drill involves hitting putts at different speeds to learn how to read and adjust for the speed of the putt. You can use a  system


: 